In [13]:
import pandas as pd
from pyspark.sql.functions import col
from datetime import datetime
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql.functions import datediff, to_date, lit
import pyspark.sql.functions as F
import matplotlib

No module named matplotlib
Traceback (most recent call last):
ImportError: No module named matplotlib



In [2]:
spark.version 

u'2.3.1'

In [215]:
df = spark.read.csv('s3n://batalhadedadositau/saude/HSL/{}'.format(x),sep=';',header=True)
df.coalesce(1).write.csv('./batalhadedadositau.csv',header=True)

In [2]:
path = ['AutIntHos-DataSUS-2013-2018.csv']
# ,'Obitos-DataSUS-2013-2016.csv'
for x in path:
    print x
    df = spark.read.csv('s3n://batalhadedadositau/saude/HSL/{}'.format(x),sep=';',header=True)
    print 'quantidade de variaveus'
    df.count()
    for column in df.columns:
        if x == 'CID_PRINCIPAL':
            df.select(x).filter(df[x] == '10').show()
#         df.groupby(column).count().show()
    df.printSchema()

AutIntHos-DataSUS-2013-2018.csv
quantidade de variaveus
354284
root
 |-- AIH_ID: string (nullable = true)
 |-- AIH_TIPO: string (nullable = true)
 |-- PROCESSAMENTO_ANO: string (nullable = true)
 |-- PROCESSAMENTO_MES: string (nullable = true)
 |-- LEITO_TIPO: string (nullable = true)
 |-- PACIENTE_CEP: string (nullable = true)
 |-- PACIENTE_MUNICIPIO: string (nullable = true)
 |-- PACIENTE_UF: string (nullable = true)
 |-- PACIENTE_DATA_NASCIMENTO: string (nullable = true)
 |-- PACIENTE_SEXO: string (nullable = true)
 |-- UTI_QTD_DIAS: string (nullable = true)
 |-- UTI_TIPO: string (nullable = true)
 |-- DIARIA_INTERMEDIARIA: string (nullable = true)
 |-- DIARIA_ACOMPANHANTE: string (nullable = true)
 |-- DIARIA: string (nullable = true)
 |-- PROCEDIMENTO_SOLICITADO_ID: string (nullable = true)
 |-- PROCEDIMENTO_SOLICITADO_DESC: string (nullable = true)
 |-- PROCEDIMENTO_REALIZADO_ID: string (nullable = true)
 |-- PROCEDIMENTO_REALIZADO_DESC: string (nullable = true)
 |-- VALOR_SERVIC

# Transformation date

In [5]:
import datetime
def date_diff(date):
    # Data final
    try:
        today = datetime.date.today()
        date = date.split('-')
        return 2018 - int(date[0])
        
    except:
        return 0
    
    # Calculo da quantidade de dias
    
def date_format(date):
    if date:
        #PACIENTE_DATA_NASCIMENTO
        ano = date[:4]
        mes = date[4:6]
        dia = date[6:]
    else:
        ano = '01'
        mes = '01'
        dia = '2001'
    return '{}-{}-{}'.format(ano,mes,dia)
date_format_udf = udf(date_format,StringType())
date_diff_udf = udf(date_diff,IntegerType())

# Filtro por uf

In [6]:
df = df.withColumn('PACIENTE_DATA_NASCIMENTO',date_format_udf(df['PACIENTE_DATA_NASCIMENTO']))
df = df.withColumn('IDADE',date_diff_udf(df['PACIENTE_DATA_NASCIMENTO']))

# GroupBy

In [246]:
df.select('CNES_UF','PACIENTE_DATA_NASCIMENTO','idade').show(300)

+-------+------------------------+-----+
|CNES_UF|PACIENTE_DATA_NASCIMENTO|idade|
+-------+------------------------+-----+
|     BA|              1991-03-15|   27|
|     BA|              1981-12-13|   37|
|     BA|              1990-04-25|   28|
|     BA|              1972-05-03|   46|
|     BA|              1973-09-09|   45|
|     BA|              1935-02-02|   83|
|     BA|              1943-02-10|   75|
|     BA|              1963-05-15|   55|
|     BA|              1957-07-17|   61|
|     BA|              1947-01-19|   71|
|     BA|              1949-10-16|   69|
|     BA|              1970-08-01|   48|
|     BA|              1930-01-22|   88|
|     BA|              1963-10-30|   55|
|     BA|              1940-09-28|   78|
|     BA|              1968-02-10|   50|
|     BA|              1972-12-11|   46|
|     BA|              1951-10-03|   67|
|     BA|              1963-04-28|   55|
|     BA|              1975-04-14|   43|
|     BA|              1941-11-02|   77|
|     BA|       

In [247]:
df.groupby(['CNES_UF']).count().sort(col("count").desc()).show(truncate=False)

+-------+------+
|CNES_UF|count |
+-------+------+
|null   |184394|
|MG     |43018 |
|PR     |24594 |
|BA     |20207 |
|PE     |19073 |
|CE     |10074 |
|ES     |9959  |
|GO     |7583  |
|DF     |4916  |
|PB     |4774  |
|MA     |4405  |
|PI     |4183  |
|MS     |3781  |
|MT     |3619  |
|AL     |3520  |
|PA     |3447  |
|AM     |1942  |
|AC     |442   |
|AP     |353   |
+-------+------+

# Descrição dos CID's

In [248]:
df.groupby(['CID_PRINCIPAL','CID_PRINCIPAL_DESC']).count().sort(col("count").desc()).show(truncate=False)

+-------------+--------------------------------------------------+------+
|CID_PRINCIPAL|CID_PRINCIPAL_DESC                                |count |
+-------------+--------------------------------------------------+------+
|C509         |C50.9 Mama NE                                     |133901|
|C508         |C50.8 Lesao invasiva da mama                      |97381 |
|C500         |C50.0 Mamilo e areola                             |33175 |
|C501         |C50.1 Porcao central da mama                      |26828 |
|C504         |C50.4 Quadrante super externo da mama             |25912 |
|C506         |C50.6 Porcao axilar da mama                       |11685 |
|C503         |C50.3 Quadrante infer interno da mama             |9098  |
|C502         |C50.2 Quadrante super interno da mama             |5475  |
|C505         |C50.5 Quadrante infer externo da mama             |4661  |
|null         |null                                              |969   |
|N648         |N64.8 Outr transt espec

In [249]:
# CID_NOTIFICACAO
df.groupby(['CID_PRINCIPAL','CID_NOTIFICACAO']).count().sort(col("count").desc()).show()

+-------------+---------------+------+
|CID_PRINCIPAL|CID_NOTIFICACAO| count|
+-------------+---------------+------+
|         C509|           null|133901|
|         C508|           null| 97380|
|         C500|           null| 33173|
|         C501|           null| 26828|
|         C504|           null| 25912|
|         C506|           null| 11685|
|         C503|           null|  9098|
|         C502|           null|  5475|
|         C505|           null|  4661|
|         null|           null|   969|
|         N648|           null|   609|
|         A419|           null|   258|
|          J90|           null|   242|
|         J189|           null|   133|
|         A488|           null|   115|
|         D486|           null|   113|
|         C773|           null|   100|
|         I878|           null|    99|
|         J960|           null|    94|
|         A499|           null|    92|
+-------------+---------------+------+
only showing top 20 rows

In [250]:
df.groupby('CID_SECUNDARIA').count().sort(col("count").desc()).show()

+--------------+------+
|CID_SECUNDARIA| count|
+--------------+------+
|          null|333720|
|          C509|  5792|
|          C508|  3436|
|           N63|  1971|
|          C506|   752|
|           I10|   728|
|          C504|   463|
|          C501|   454|
|          C773|   389|
|          C500|   292|
|          Z515|   288|
|          R060|   235|
|           E86|   209|
|          N648|   194|
|          D630|   193|
|           J90|   166|
|          Y838|   166|
|          C795|   160|
|          C780|   148|
|          Z439|   139|
+--------------+------+
only showing top 20 rows

# CID Associado ao valor do serviço hospitalar

In [251]:
df.groupby('CID_PRINCIPAL').agg({'VALOR_SERVICO_HOSPITALAR':'sum'}).sort(col("sum(VALOR_SERVICO_HOSPITALAR)").desc()).show(truncate=False)

+-------------+-----------------------------+
|CID_PRINCIPAL|sum(VALOR_SERVICO_HOSPITALAR)|
+-------------+-----------------------------+
|C509         |1.5310932949998862E8         |
|C508         |1.4983713644999343E8         |
|C501         |4.7814564910001345E7         |
|C504         |4.7649161469999924E7         |
|C500         |4.417092167999957E7          |
|C503         |1.7357095190000046E7         |
|C506         |1.676633930000022E7          |
|C502         |8686476.569999922            |
|C505         |7902171.939999953            |
|A419         |823975.0000000001            |
|N648         |431720.9300000009            |
|J90          |275189.4199999998            |
|M518         |218342.72                    |
|D486         |204868.7200000001            |
|A488         |172123.37000000005           |
|C773         |164956.79                    |
|A499         |148682.09999999998           |
|J960         |142423.16                    |
|J189         |140818.51          

In [252]:
# MORTE
df.groupby('MORTE').agg({'CID_PRINCIPAL':'count'}).sort(col("count(CID_PRINCIPAL)").desc()).show(truncate=False)

+-----+--------------------+
|MORTE|count(CID_PRINCIPAL)|
+-----+--------------------+
|0    |322821              |
|1    |30183               |
|null |311                 |
+-----+--------------------+

In [253]:
df.groupby('CID_PRINCIPAL','idade').count().sort(col("count").desc()).show(100)

+-------------+-----+-----+
|CID_PRINCIPAL|idade|count|
+-------------+-----+-----+
|         C509|   54| 3987|
|         C509|   53| 3842|
|         C509|   56| 3840|
|         C509|   55| 3822|
|         C509|   58| 3728|
|         C509|   60| 3708|
|         C509|   52| 3650|
|         C509|   59| 3628|
|         C509|   63| 3575|
|         C509|   51| 3575|
|         C509|   61| 3573|
|         C509|   62| 3554|
|         C509|   57| 3521|
|         C509|   64| 3463|
|         C509|   50| 3458|
|         C509|   65| 3237|
|         C509|   66| 3189|
|         C509|   49| 3180|
|         C508|   54| 3013|
|         C509|   67| 3011|
|         C509|   68| 3008|
|         C508|   55| 2951|
|         C509|   48| 2948|
|         C508|   53| 2896|
|         C509|   47| 2840|
|         C508|   56| 2834|
|         C509|   46| 2815|
|         C508|   52| 2803|
|         C508|   58| 2788|
|         C508|   57| 2783|
|         C508|   50| 2768|
|         C509|   70| 2674|
|         C508|   59

In [7]:
df.select('INTERNACAO_DATA').show()

+---------------+
|INTERNACAO_DATA|
+---------------+
|       20130110|
|       20130108|
|       20130113|
|       20121025|
|       20121217|
|       20121128|
|       20121217|
|       20121212|
|       20130113|
|       20130114|
|       20121224|
|       20121224|
|       20121226|
|       20121123|
|       20121208|
|       20121211|
|       20121106|
|       20121115|
|       20121216|
|       20121001|
+---------------+
only showing top 20 rows

In [7]:
df = df.withColumn('INTERNACAO_DATA',  F.to_timestamp('INTERNACAO_DATA','yyyyMMdd')) 

In [8]:
df_pd = df.toPandas()

In [9]:
df_pd.index = df_pd.INTERNACAO_DATA

In [10]:
x = df_pd.resample('Y').count()[['VALOR_SERVICO_HOSPITALAR']]